# 概要
* 複数インデックスのマージにいよいよトライする
* treeは自動で要約つけてくれるの？？めっちゃ便利やん
* 訳としては完璧やん...
* num_outputは2048も対応している→ダイでエラー発生
* 親の要約が変なのが惜しい
* b_indexがおかしい
* そもそもnodeにembeddingがある場合とない場合がある

# 参考
* そもそもtree構造が前提なのでtreeのindexを確認する
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html
* https://note.com/npaka/n/n5d4ff4380055

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [5]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

#documents = SimpleDirectoryReader('story/dai').load_data()
documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024+512
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
#d_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
#b_index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1


In [6]:
documents = SimpleDirectoryReader('story/dai').load_data()
d_index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
INFO:root:> Building index from nodes: 2 chunks
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to use only the information provided. Try to include as many key details as possible.\\n\\n\\n\\u5e8f\\u7ae0\\uff08\\u30c7\\u30eb\\u30d1! \\u30a4\\u30eb\\u30a4\\u30eb! / \\u30c0\\u30a4\\u7206\\u767a!!!\\uff09\\n\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\\u52c7\\u8005\\u7387\\u3044\\u308b\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u6226\\u3044\\u306f\\u3001\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u3067\\u7d42\\u308f\\u308a\\u3001\\u9b54\\u738b\\u306e\\u914d\\u4e0b\\u3067\\u3042\\u3063\\u305f\\u30e2\\u30f3\\u30b9\\u30bf\\u30fc\\u3082\\u9b54\\u738b\\u306e\\u652f\\u914d\\u304b\\u3089\\u89e3\\u304d\\u653e\\u305f\\u308c\\u300

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=26449 request_id=4484c44724826360dd02a637f2d74264 response_code=200
DEBUG:root:
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=25070 request_id=a91c80da17e280a8f5c159c99944df4f response_code=200
DEBUG:root:
レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと宇宙空間で繰り広げられる両者の最後の激闘は、ダイの勝利で終わる。地上に帰還したダイが仲間たちに迎えられ、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかった。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to

In [7]:
d_index.docstore.docs#['38f1d4b1-3314-4635-a146-82951b12f846'].all_nodes#.all_nodes

{'fc284957-9a52-485f-9f2a-9a6d9c1ade64': IndexGraph(text=None, doc_id='fc284957-9a52-485f-9f2a-9a6d9c1ade64', embedding=None, extra_info=None, all_nodes={0: Node(text='序章（デルパ! イルイル! / ダイ爆発!!!）\n魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。\n\nそれから十数年後', doc_id='98d3da4b-11ae-45ab-8653-3a75aca00b32', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='31d95fba-1e02-482e-997b-5034c4364ec5', node_info=None), 1: Node(text='多数のモンスターが住む怪物の島・デルムリン島に赤子の時に漂着し鬼面道士のブラスに育てられた少年ダイは、島唯一の人間として、ゴールデンメタルスライムのゴメちゃんを始めとする友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中でも何度かの冒険や騒動もあり、その過程でダイはパプニカ王国王女のレオナと知り合う', doc_id='5111c93d-487f-433a-b3e5-d70d72e82beb', embedding=None, extra_info=None, index=1, child_indices=set(), ref_doc_id='31d95fba-1e02-482e-997b-5034c4364ec5', node_info=None), 2: Node(text='\n\nダイの旅立ち\nある日、ゴメちゃん以外の島のモンスターたちが凶暴化。辛うじて理性を保っているブラスは魔王が復活したことを察する。そこに勇者の家庭教師を名乗る謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはレオナからダイの事を聞

In [8]:
d_index.docstore.docs['fc284957-9a52-485f-9f2a-9a6d9c1ade64'].root_nodes#.all_nodes

{21: Node(text='\n十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。', doc_id='63c191d1-2a3e-4655-8f51-dd09ea97bd5d', embedding=None, extra_info=None, index=21, child_indices={0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, ref_doc_id=None, node_info=None),
 22: Node(text='\nレオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡

In [9]:
d_index.set_text="""
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。
レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと宇宙空間で繰り広げられる両者の最後の激闘は、ダイの勝利で終わる。地上に帰還したダイが仲間たちに迎えられ、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかった。
"""

In [18]:
d_index.docstore.docs['fc284957-9a52-485f-9f2a-9a6d9c1ade64'].text="""
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。
レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと宇宙空間で繰り広げられる両者の最後の激闘は、ダイの勝利で終わる。地上に帰還したダイが仲間たちに迎えられ、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかった。
"""

In [19]:
d_index.docstore

DocumentStore(docs={'fc284957-9a52-485f-9f2a-9a6d9c1ade64': IndexGraph(text='\n十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\nレオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと

In [80]:
b_index = GPTTreeIndex.load_from_disk('dragon_tree.json', prompt_helper=prompt_helper, llm_predictor=llm_predictor)

In [32]:
#b_index.set_text("ドラゴンボールを巡って悟空が冒険します")

In [33]:
b_index.docstore

DocumentStore(docs={'f3f594ba-5b3d-4463-8b33-4bab69e8788b': IndexGraph(text='\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。\n\nピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。\n\nサイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を守るため、宇宙へと旅立つ。\n\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロ

In [68]:
all_index = GPTListIndex([b_index, d_index])

DEBUG:root:> Adding chunk: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:root:> Adding chunk: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ...
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens


In [35]:
all_index.query("キルバーンは最後どうなりましたか？")

DEBUG:root:> Searching in chunk: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:root:> Searching in chunk: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4282 request_id=70c693496f4f9fcf5c922d51c9b82859 response_code=200
DEBUG:root:
キルバーンは最後、悟空とウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じることで終わりました。
DEBUG:root:> Initial response: 
キルバーンは最後、悟空とウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じることで終わりました。
DEBUG:root:> Refine context: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u308a\\u307e\\u3057\\u305f\\u304b\\uff1f\\nWe have provided an existing answer: \\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u6700\\u5f8c\\u3001\\u609f\\u7a7a\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=5839 request_id=fc94abd982d6549b7bfeb95a948bb98a response_code=200
DEBUG:root:

キルバーンは最後、ダイとポップが核晶を空高く運ぶ中で、土壇場でダイがポップを地上に戻し、一人爆発の閃光の中に消えてしまいました。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、物語は幕を閉じることで終わりました。
DEBUG:root:> Refined response: 

キルバーンは最後、ダイとポップが核晶を空高く運ぶ中で、土壇場でダイがポップを地上に戻し、一人爆発の閃光の中に消えてしまいました。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、物語は幕を閉じることで終わりました。
INFO:root:> [query] Total LLM token usage: 5323 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\n\nキルバーンは最後、ダイとポップが核晶を空高く運ぶ中で、土壇場でダイがポップを地上に戻し、一人爆発の閃光の中に消えてしまいました。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、物語は幕を閉じることで終わりました。', source_nodes=[SourceNode(source_text='\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。\n\nピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。\n\nサイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を守るため、宇宙へと旅立つ。\n\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下

In [36]:
all_index.query("魔人ブウはどのように倒されましたか？")

DEBUG:root:> Searching in chunk: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:root:> Searching in chunk: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4299 request_id=adc9e3ab949d51af9cace03341fdc2eb response_code=200
DEBUG:root:
魔人ブウは、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって完全に消滅しました。
DEBUG:root:> Initial response: 
魔人ブウは、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって完全に消滅しました。
DEBUG:root:> Refine context: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u3069\\u306e\\u3088\\u3046\\u306b\\u5012\\u3055\\u308c\\u307e\\u3057\\u305f\\u304b\\uff1f\\nWe have provided an existing answer: \\n\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5354\\u5

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3690 request_id=aedca5beeecfb3bb7c0f0f6a1bb24cbd response_code=200
DEBUG:root:
魔人ブウは、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって、ダイ一行の勇者パーティの力によって完全に消滅しました。
DEBUG:root:> Refined response: 
魔人ブウは、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって、ダイ一行の勇者パーティの力によって完全に消滅しました。
INFO:root:> [query] Total LLM token usage: 5298 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\n魔人ブウは、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって、ダイ一行の勇者パーティの力によって完全に消滅しました。', source_nodes=[SourceNode(source_text='\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。\n\nピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。\n\nサイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を守るため、宇宙へと旅立つ。\n\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大

### 階層構造の場合は、modeは再帰にし、詳細な条件はquery_configsで指定する

In [71]:
query_configs = [
    {
        "index_struct_type": "list",
        "query_mode": "embedding",
        "query_kwargs": {
             #"child_branch_factor": 1
             "similarity_top_k": 1
        }
    }
]

In [72]:
#  recursiveではquery_configs 必須
all_index.query("魔人ブウはどのように倒されましたか？", mode="recursive", query_configs=query_configs)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u3069\\u306e\\u3088\\u3046\\u306b\\u5012\\u3055\\u308c\\u307e\\u3057\\u305f\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=107 request_id=9469c41ca6f0e20b1333d6b2e6663769 response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8423
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=120 request_id=d0148da4fdf54d00bff86ea772217e3e response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u3092\\u5012\\u3057\\u305f\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u306b\\u3088\\u308a\\u3001\\u9b54\\u738b\\u306e\\u652f\\u914d\\u304b\\u3089\\u30e2\\u30f3\\u30b9\\u30bf\\u30fc\\u304c\\u89e3\\u304d\\u653e\\u305f\\u308c\\u3001\\u4e16\\u754c\\u306b\\u5e73\\u548c\\u304c\\u8a2a\\u308c\\u305f\\u3002\\u305d\\u306e\\u5f8c\\u3001\\u8d64\\u5b50\\u306e\\u6642\\u306b\\u6f02\\u7740\\u3057\\u305f\\u30c7\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8423
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=62 request_id=0b47622aad7071c2ae086ddca67328e4 response_code=200
DEBUG:root:> Top 1 nodes:

DEBUG:root:
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。
レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 548
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1927 request_id=84d07cbed672095e477f9459acfccf89 response_code=200
DEBUG:root:
ANSWER: 2. This summary was selected because it describes the final battle between Dai and Majin Buu, which resulted in Dai's victory. It also mentions Dai's transformation into a Dragon Knight and Buu's transformation into the Kigan King, which are both relevant to the question.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れた

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 608
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2809 request_id=0c83d9d7bc67544e3958e50c1ae3da9c response_code=200
DEBUG:root:
ANSWER: 8. This summary was selected because it describes the final battle between Dai and Majin Buu, in which Dai transforms into a Dragon-Demon and Majin Buu releases his true power, the Demon Eye, to become the Demon Eye King. It also mentions that Dai defeats Buu with the power of the Dragon Emblem he inherited from his father, Balan.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力を蓄えていく

(2) 一方、失態を重ねて奮起したハドラーは妖魔司

Response(response='魔人ブウは、ダイが竜の魔人となり、バーンが鬼眼王と化した激闘の末に倒されました。', source_nodes=[SourceNode(source_text='\n十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\nレオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御

In [76]:
b_index.query("魔人ブウはどのように倒されましたか？")

INFO:root:> Starting query: 魔人ブウはどのように倒されましたか？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a2\\u3059\\u65c5\\u306b\

DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。  ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される

(2)  悟空は孫悟飯を助けるため、宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる

(3)  セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセルに

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 464
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2741 request_id=5c046f67dde4ed2327fb93e22a8bba95 response_code=200
DEBUG:root:
ANSWER: 3. Because this summary states that Son Gohan was able to awaken to Super Saiyan 2 and, with the help of the phantom of his father Goku, was able to fire a Kamehameha wave and defeat Cell.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する

(2) 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす

(3) だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破

(4) セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ

(5)   魔人ブウ編 セルゲ

Response(response='魔人ブウは悟飯が超サイヤ人2へと覚醒し、父・悟空の幻影と共にかめはめ波を放ち、それによって倒されました。', source_nodes=[SourceNode(source_text='だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破', doc_id='98733fdf-2be2-421f-8178-2ccdb0f69918', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [81]:
b_index.query("魔人ブウは誰が倒しましたか？", mode="default")

INFO:root:> Starting query: 魔人ブウは誰が倒しましたか？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a2\\u3059\\u65c5\\u306b\\u51

DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。  ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される

(2)  悟空は孫悟飯を助けるため、宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる

(3)  セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセルに

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 427
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1728 request_id=a3ce397cd054e1159be97e91f57e5502 response_code=200
DEBUG:root:
ANSWER: 3. Because this summary states that Son Gohan, with the help of the phantom of his father Goku, was able to defeat Cell by releasing a Kamehameha wave.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する

(2) 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす

(3) だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破

(4) セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ

(5)   魔人ブウ編 セルゲームより約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った

Response(response='悟飯が魔人ブウを倒しました。', source_nodes=[SourceNode(source_text='だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破', doc_id='98733fdf-2be2-421f-8178-2ccdb0f69918', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [82]:
b_index.query("魔人ブウは誰が倒しましたか？", mode="embedding")

INFO:root:> Starting query: 魔人ブウは誰が倒しましたか？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u8ab0\\u304c\\u5012\\u3057\\u307e\\u3057\\u305f\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=49 request_id=a8d33d140cd4152bca007f563130107c response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u3

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8423
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=117 request_id=12d900be9ebdd7d20acd3a894211a4f9 response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u609f\\u7a7a\\u306f\\u5b6b\\u609f\\u98ef\\u3092\\u52a9\\u3051\\u308b\\u305f\\u3081\\u3001\\u5bbf\\u6575\\u30d4\\u30c3\\u30b3\\u30ed\\u3068\\u624b\\u3092\\u7d44\\u307f\\u3001\\u81ea\\u3089\\u306e\\u547d\\u3068\\u5f15\\u304d\\u63db\\u3048\\u306b\\u30e9\\u30c7\\u30a3\\u30c3\\u30c4\\u3092\\u5012\\u3059\\u3002\\u305d\\u306e\\u5f8c\\u3001\\u7d041\\u5e74\\u5f8c\\u306b\\u306f\\u3055\\u3089\\u306b\\u5f37\\u529b\\u306a\\u30b5\\u30a4\\u30e4\\u4eba\\u305f\\u3061\\u304c\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8421
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=45 request_id=8628b56c77d54718d4fcdc594d5fdae2 response_code=200
DEBUG:root:>[Level 0] Node [3] Summary text:  セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。  約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌し、悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、苦戦を強いられる。悟空とベジータが界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。  10年後、悟空

DEBUG:openai:api_version=None data='{"input": ["\\u5fa9\\u6d3b\\u3057\\u305f\\u9b54\\u4eba\\u30d6\\u30a6\\u306b\\u3088\\u308a\\u609f\\u98ef\\u3084\\u30d9\\u30b8\\u30fc\\u30bf\\u304c\\u5012\\u3055\\u308c\\u3001\\u609f\\u7a7a\\u306f\\u3042\\u306e\\u4e16\\u306b\\u5e30\\u3063\\u305f\\u305f\\u3081\\u3001\\u5730\\u7403\\u306e\\u547d\\u904b\\u306f\\u609f\\u7a7a\\u306e\\u6b21\\u7537\\u30fb\\u5b6b\\u609f\\u5929\\u3068\\u5c11\\u5e74\\u30c8\\u30e9\\u30f3\\u30af\\u30b9\\u306e\\u5e7c\\u3044\\u4e8c\\u4eba\\u306b\\u8a17\\u3055\\u308c\\u308b\\u3002  \\u4e00\\u65b9\\u3001\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u69d8\\u3005\\u306a\\u4eba\\u9593\\u3068\\u306e\\u51fa\\u4f1a\\u3044\\u304b\\u3089\\u3088\\u308a\\u90aa\\u60aa\\u3067\\u5f37\\u529b\\u306a\\u9b54\\u4eba\\u3078\\u3068\\u5909\\u8c8c"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8421
DEBUG:openai:message='OpenAI API respo

Response(response='魔人ブウは孫悟天と少年トランクスの二人によって倒されました。', source_nodes=[SourceNode(source_text='復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。\n\n一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌', doc_id='98733fdf-2be2-421f-8178-2ccdb0f69918', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [84]:
b_index.query("魔人ブウは誰が倒しましたか？", child_branch_factor=2)

INFO:root:> Starting query: 魔人ブウは誰が倒しましたか？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 3), where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a2\\u3059\\u65c5\\u306b\\u5

DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3), where each item in the list corresponds to a summary.
---------------------
(1)  孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。  ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される

(2)  悟空は孫悟飯を助けるため、宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる

(3)  セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセル

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 702
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3497 request_id=487bc9cf02c62add2703cb093855d77e response_code=200
DEBUG:root:
ANSWER: 9, 10. 
These summaries were selected because they both mention the character of Trunks, who is the one who defeats Majin Buu in the Dragon Ball series. Summary 9 mentions Trunks as the mysterious Super Saiyan who defeats Frieza and his army, while Summary 10 mentions Trunks as the one who reveals his identity as the son of Bulma and Vegeta and warns Goku of the two artificial humans who will bring despair in three years.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10), where each item in the list corresponds to a summary.
---------------------
(1) さらわれた孫悟飯を助けるため悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 485
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1663 request_id=39368aa43522f42dd45f7eef6d0949a1 response_code=200
DEBUG:root:

The mysterious Super Saiyan, Trunks, who revealed to Goku that he was the son of Bulma and Vegeta from the future, eventually defeated Majin Buu after a series of warriors appeared, far surpassing Goku's expectations.
DEBUG:root:>[Level 1] Refine prompt: The original question is as follows: 魔人ブウは誰が倒しましたか？
We have provided an existing answer: 
The mysterious Super Saiyan defeated Majin Buu.
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。悟空らの想定を遥かに超えた戦士が続々と現れた

The mysterious Super Saiyan, Trunks, who revealed to Goku that he was the son of Bulma

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=7257 request_id=d129c92bdef007dc6732c7f6adba8db9 response_code=200
DEBUG:root:
ANSWER: 3 and 9. 
The summary in choice 3 states that "悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破" which implies that 悟飯 was the one who defeated セル. The summary in choice 9 states that "ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する" which implies that the combination of 悟空, ベジータ, and the energy of the Dragon Ball was used to defeat 魔人ブウ.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10), where each item in the list corresponds to a summary.
---------------------
(1)   人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する

(2) 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4018 request_id=b7091f3a460e21a2b2370feca5428513 response_code=200
DEBUG:root:

魔人ブウは悟飯が超サイヤ人2へと覚醒し、父・悟空の幻影と共にかめはめ波を放ち、ドラゴンボールの協力と地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉を使って倒しました。
DEBUG:root:>[Level 1] Refine prompt: The original question is as follows: 魔人ブウは誰が倒しましたか？
We have provided an existing answer: 
魔人ブウは悟飯が超サイヤ人2へと覚醒し、父・悟空の幻影と共にかめはめ波を放ちセルを撃破して倒しました。
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
危機に陥った悟飯らを救うため現世に舞い戻った悟空とベジータは、界王神界で真の姿となった魔人ブウとの最終決戦に臨む。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する


魔人ブウは悟飯が超サイヤ人2へと覚醒し、父・悟空の幻影と共にかめはめ波を放ち、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉を使って倒しました。
------------
Given the new context, refine the original answer to better answer the question. If t

DEBUG:root:>[Level 0] Refine prompt: The original question is as follows: 魔人ブウは誰が倒しましたか？
We have provided an existing answer: 

The mysterious Super Saiyan, Trunks, who revealed to Goku that he was the son of Bulma and Vegeta from the future, eventually defeated Majin Buu after a series of warriors appeared, far surpassing Goku's expectations.
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------

セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。

約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌し、悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、苦戦を強いられる。悟空とベジータが界王神界で真の姿となった

Response(response="The mysterious Super Saiyan, Trunks, who revealed to Goku that he was the son of Bulma and Vegeta from the future, eventually defeated Majin Buu after a series of warriors appeared, far surpassing Goku's expectations. With the help of Goku's phantom, Trunks achieved Super Saiyan 2 and used a Kamehameha wave to weaken Buu. With the help of the Dragon Balls and the energy of the people of Earth, the Nameless Planet, and the Other World, a powerful Spirit Bomb was created and used to finally defeat Majin Buu.", source_nodes=[SourceNode(source_text='\n\n人造人間・セル編\nナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される', doc_id='98733fdf-2be2-421f-8178-2ccdb0f69918', extra_info=None, node_info=None, similarity=None), SourceNode(source_text='トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。悟空らの想定を遥かに超えた戦士が続々と現れた', doc_id='98733fdf-2be2-421f-8178-2ccdb0f69918', extra_info=None, node_info=None, similarity=None),

In [87]:
all_index = GPTSimpleVectorIndex([b_index, d_index])

DEBUG:root:> Adding chunk: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3053\\u3068\\u3067\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u3092\\u63a2\\u3059\\u65c5\\u306b\\u51fa\\u308b\\u3002\\u305d\\u306e\\u9053\\u4e2d\\u3067\\u30a6\\u30fc\\u30ed\\u30f3\\u3084\\u30e4

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8423
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=128 request_id=6ba2cc2f83f5ff0f976153020a0a667f response_code=200
DEBUG:root:> Adding chunk: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [" \\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u3092\\u5012\\u3057\\u305f\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u306b\\u3088\\u308a\\u3001\\u9b54\\u738b\\u306e\\u652f\\u914d\\u304b\\u3089\\u30e2\\u30f3\\u30b9\\u30bf\\u30fc\\u304c\\u89e3\\u304d\\u653e\\u305f\\u308c\\u3001\\u4e16\\u754c\\u306b\\u5e73\\u548c\\u304c\\u8a2a\\u308c\\u305f\\u3002\\u305d\\u306e\\u5f8c

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8423
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=64 request_id=ec86b1a60a2ce57c0e04ddf538b1749b response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4854 tokens


In [93]:
all_index.docstore

DocumentStore(docs={'f3f594ba-5b3d-4463-8b33-4bab69e8788b': IndexGraph(text='\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。\n\nピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。\n\nサイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を守るため、宇宙へと旅立つ。\n\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロ

In [90]:
query_configs = [
    {
        "index_struct_type": "list",
        "query_mode": "embedding",
        "query_kwargs": {
             "similarity_top_k": 1
        }
    }
]

In [91]:
all_index.query("魔人ブウは誰が倒しましたか？", mode="recursive", query_configs=query_configs)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u9b54\\u4eba\\u30d6\\u30a6\\u306f\\u8ab0\\u304c\\u5012\\u3057\\u307e\\u3057\\u305f\\u304b\\uff1f"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=63 request_id=5b78619af106c8567269f72e6f62d859 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node 794da275-b487-4bec-a3b5-79d589cb23c5] [Similarity score:             0.829434] 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイ...
DEBUG:root:> Searching in chunk: 
十数

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 482
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2134 request_id=21a246a2dd6150ac28c2fc8ab18fbb1d response_code=200
DEBUG:root:
ANSWER: 3. This summary was selected because it mentions Dai, who is the one who defeated Majin Buu. It also mentions Pop, who is Dai's companion, and the magical gem on Dai's sword, which is connected to his life.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデル

Response(response='魔人ブウは、ロン・ベルクとポップが共同で倒しました。', source_nodes=[SourceNode(source_text='\n十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、モンスターの友達と共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団を打ち破り、レオナの救出にも成功した。最後に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\nレオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続け

In [101]:
b_index.docstore.docs["f3f594ba-5b3d-4463-8b33-4bab69e8788b"].root_nodes

{30: Node(text='\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。\n\nピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。\n\nサイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を守るため、宇宙へと旅立つ。\n\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間た